In [1]:
import csv
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing

In [2]:
def get_river_predictors(fname, pr_list=None, y_name='h_max'):
    pr_arr = []
    y_arr = []
    with open(fname, newline='') as f:
        reader = csv.DictReader(f, delimiter=';')
        for row in reader:
            pr_arr_row = []
            for col in pr_list:
                pr_arr_row.append(row[col])
                #print(f'{col}: {row[col]}', end='; ')
            pr_arr.append(pr_arr_row)
            y_arr.append(row[y_name])
    X = np.asarray(pr_arr, dtype=np.float32)
    y = np.asarray(y_arr, dtype=np.float32)
    return X, y

#### Среднеквадратическое отклонение sigma

In [3]:
def get_sigma(hydropost):
    sigma = {
        'Неман-Белица': 93,
    }
    return sigma[hydropost]

#### Допустимая погрешность прогноза sigma_dop

In [4]:
def get_sigma_dop(sigma):
    return 0.674 * sigma

#### Сумма, средний, высший, низший уровни

In [5]:
def get_sum(h_max):
    return np.sum(h_max)
    
def get_avg(h_max):
    return np.mean(h_max)
    
def get_max(h_max):
    return np.amax(h_max)
    
def get_min(h_max):
    return np.amin(h_max)

#### Среднеквадратическая погрешность S_climat

In [6]:
def get_s_climat(h_max):
    # Среднеквадратическая погрешность
    return np.std(h_max, ddof=1)

#### Критерий эффективности метода прогнозирования климатический S/sigma

In [7]:
def get_criterion_climat(s_climat, sigma):
    return s_climat / sigma

#### Климатическая обеспеченность Pk

In [8]:
def get_pk(h_max, sigma_dop):
    avg_level = np.mean(h_max)
    diff = np.abs(h_max - avg_level)
    trusted_values = diff[diff <= sigma_dop]
    return trusted_values

### Формирование набора данных

In [9]:
pr_list = ['s_2802', 's_max', 'h', 'x', 'x1', 'x2', 'x3', 'x4', 'xs']

In [10]:
X, y = get_river_predictors('data/Неман-Белица-2010.csv', pr_list=pr_list)

### Вычисление статистики

In [11]:
# Среднеквадратическое отклонение sigma
sigma = get_sigma('Неман-Белица')
print(f'Среднеквадратическое отклонение sigma = {sigma}')

# Допустимая погрешность прогноза sigma_dop
sigma_dop = get_sigma_dop(sigma)
print(f'Допустимая погрешность прогноза sigma_dop = {sigma_dop}')

# Статистика максимального уровня
sum_level = get_sum(y)
avg_level = get_avg(y)
max_level = get_max(y)
min_level = get_min(y)
print(f'Сумма {sum_level}, Средний {avg_level}, Высший {max_level}, Низший {min_level}')

# Среднеквадратическая погрешность климатическая S
s_climat = get_s_climat(y)
print(f'Среднеквадратическая погрешность климатическая: {s_climat}')

# Критерий эффективности метода прогнозирования климатический S/sigma
criterion_climat = get_criterion_climat(s_climat, sigma)
print(f'Критерий эффективности метода прогнозирования климатический S/sigma = {criterion_climat}')

# Климатическая обеспеченность Pk
pk = get_pk(y, sigma_dop)
print(pk)

Среднеквадратическое отклонение sigma = 93
Допустимая погрешность прогноза sigma_dop = 62.682
Сумма 8743.0, Средний 301.4827575683594, Высший 494.0, Низший 187.0
Среднеквадратическая погрешность климатическая: 73.22158813476562
Критерий эффективности метода прогнозирования климатический S/sigma = 0.7873289046748992
[20.482758  9.517242 55.517242  8.482758 32.482758  9.482758 52.482758
 60.517242 52.517242 59.517242 56.482758 49.482758 43.517242 37.482758
 39.517242 47.482758 44.517242 15.482758]


### Обучающий набор

In [12]:
test_qnt = 5

In [13]:
X_train = X[:-test_qnt]
X_test = X[-test_qnt:]

In [14]:
y_train = y[:-test_qnt]
y_test = y[-test_qnt:]

## Этап нормализации

In [15]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)


***Этап выбора предикторов 

In [16]:
regr = linear_model.LinearRegression()
regr = regr.fit(X_train, y_train)
y_predicted = regr.predict(X_test)
print(y_predicted)

[391.6121  270.77252 241.98505 262.59338 394.25763]


In [17]:
regr_sc = linear_model.LinearRegression()
regr_sc = regr.fit(X_train_sc, y_train)
y_predicted = regr.predict(X_train_sc)
print(y_predicted)

[318.13388 312.99374 449.7868  337.92435 290.10712 322.85605 285.67328
 242.60152 333.3111  344.278   305.12723 356.0053  210.08038 247.47137
 195.08206 241.5033  394.0265  199.71321 391.25323 395.59488 244.62839
 266.76663 325.56165 308.52032]


In [18]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor()
regr = regr.fit(X_train, y_train)
y_predicted = regr.predict(X_train)
print(y_predicted)

[284.82 307.33 437.87 349.07 292.74 288.76 284.83 265.34 355.55 340.84
 345.06 360.23 209.77 252.15 234.68 277.45 380.47 228.85 347.82 363.12
 227.29 259.95 334.71 277.92]
